In [1]:
# https://www.reddit.com/r/pushshift/comments/ajmcc0/information_and_code_examples_on_how_to_use_the/ef012vk/

import zstandard
import os
import json
import sys
import time
import argparse
from datetime import datetime
import logging.handlers
import multiprocessing
import pandas as pd

In [2]:
depression_users = pd.read_csv("../dep_users_cohort.csv")
year_month = "2020-09"

dep_users = depression_users["author"].tolist()
time_users = depression_users.loc[depression_users["year-month"] == year_month]["author"].tolist()
len(time_users)

1794

In [3]:
yesdep_keep = []
j=0
with open(f"RS_{year_month}.zst", 'rb') as fh:
    print(f"RS_{year_month}.zst")
    dctx = zstandard.ZstdDecompressor(max_window_size=2147483648)
    with dctx.stream_reader(fh) as reader:
        previous_line = ""
        while True:
            chunk = reader.read(2**24)  # 16mb chunks
            if not chunk:
                break

            string_data = chunk.decode('utf-8')
            lines = string_data.split("\n")
            for i, line in enumerate(lines[:-1]):
                if i == 0:
                    line = previous_line + line
                object = json.loads(line)
                if object["is_self"]:
                    if object["author"] in time_users:
                        yesdep_keep.append(object)

            previous_line = lines[-1]

RS_2020-09.zst


In [4]:
print(len(yesdep_keep))
pd.DataFrame(yesdep_keep).to_csv(f"{year_month}-dep.csv")

5258


In [21]:
year_month = "2020-08"

dep_subbredits = pd.read_csv(f"{year_month}-dep.csv")

other_subreddits = dep_subbredits.loc[dep_subbredits["subreddit"]!="depression"]
other_subreddits = other_subreddits.loc[~other_subreddits["selftext"].isna()]
other_subreddits = other_subreddits.loc[~other_subreddits["selftext"].isnull()]
other_subreddits = other_subreddits.loc[other_subreddits["selftext"] != "[removed]"]
print(other_subreddits.shape)
other_subreddits = other_subreddits[["author", "is_self", "subreddit", "subreddit_subscribers", "title", "selftext"]]

other_subreddits = other_subreddits.sort_values("subreddit_subscribers", ascending=False).reset_index(drop=True)

other_subreddits["post_length"] = other_subreddits["selftext"].apply(lambda x: len(x.split(" ")))
other_subreddits = other_subreddits.loc[other_subreddits["post_length"] > 20]
print(other_subreddits.shape)

most_popular_other = other_subreddits.groupby("author").nth(0).reset_index()
most_popular_other.to_csv(f"{year_month}-yes-dep.csv")
print(most_popular_other.shape)
print(most_popular_other["subreddit"].nunique())
subreddit_dict = most_popular_other.groupby("subreddit").size().to_dict()
subreddit_keys = list(subreddit_dict.keys())

(2600, 89)
(2179, 7)
(620, 7)
334


In [22]:
nondep_keep = []
with open(f"RS_{year_month}.zst", 'rb') as fh:
    print(f"RS_{year_month}.zst")
    dctx = zstandard.ZstdDecompressor(max_window_size=2147483648)
    with dctx.stream_reader(fh) as reader:
        previous_line = ""
        while True:
            chunk = reader.read(2**24)  # 16mb chunks
            if not chunk:
                break

            string_data = chunk.decode('utf-8')
            lines = string_data.split("\n")
            for i, line in enumerate(lines[:-1]):
                if i == 0:
                    line = previous_line + line
                object = json.loads(line)


                if object["is_self"]:
                    if object["subreddit"] in subreddit_keys:
                        if object["author"] not in dep_users:
                            if len(object["selftext"].split(" ")) > 20:
                                subreddit_dict[object["subreddit"]] = subreddit_dict[object["subreddit"]] - 1
                                if subreddit_dict[object["subreddit"]] == 0:
                                    subreddit_dict.pop(object["subreddit"])
                                    subreddit_keys = list(subreddit_dict.keys())
                                nondep_keep.append(object)

            previous_line = lines[-1]

RS_2020-08.zst


In [23]:
print(len(nondep_keep))
pd.DataFrame(nondep_keep).to_csv(f"{year_month}-non-dep.csv")

615
